In [101]:
!pip install torchkge
!pip install soynlp

In [102]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import cosine_similarity
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from torchkge.models import ConvKBModel
import pickle 
from copy import deepcopy
from tqdm import tqdm 
from kge_train import train_kge
from torchkge.utils.datasets import load_fb15k

import transformers
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

from soynlp.hangle import levenshtein
from soynlp.hangle import jamo_levenshtein


torch.backends.cudnn.benchmark = True
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [103]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [104]:
HOME_DIR = "./"

In [105]:
#df = pd.read_csv("./datasets/20220920_csv/병원_triple.csv", encoding='cp949')
#df[530:550]

In [106]:
#df = pd.read_csv("./datasets/04 경찰서 병원 소방서/소방서_triple.csv", encoding='cp949')
FOLDER_NAME = "datasets/20220920_csv/"

In [107]:
path_to_file_folder = HOME_DIR+FOLDER_NAME
meta_data= ["file_name", "colum_name", "value_list"] #the meta-data should be extented based on EDA on the datasets
df_full = pd.DataFrame(columns=meta_data)
meta_data= ["head", "relation", "tail", "file_name"] #the meta-data should be extented based on EDA on the datasets
kg_full = pd.DataFrame(columns=meta_data)

for file_name in [file for file in os.listdir(path_to_file_folder)]:
    file_name = file_name
    a_file = HOME_DIR+FOLDER_NAME+file_name
    print("Reading the file: ", a_file)
    
    try:
        a_df = pd.DataFrame()
        if ".csv" in file_name:
            a_df = pd.read_csv(a_file, encoding='cp949', names=meta_data, header=None)
            a_df["file_name"] = file_name
        elif ".xlsx" in file_name:
            a_df = pd.read_excel(a_file, names=meta_data, header=None)
            a_df["file_name"] = file_name
        else:
            continue
        kg_full = kg_full.append(a_df, ignore_index=True)
        

        file_name_list = []
        colum_name_list = []
        value_list = []
        for col in a_df.columns:
            col_name = col
            col_values = list(a_df[col_name].values)
            file_name_list.append(file_name)
            colum_name_list.append(col_name)
            value_list.append(col_values)

        tmp_df = pd.DataFrame({"file_name": file_name_list,
                            "colum_name": colum_name_list,
                            "value_list": value_list})

        df_full = df_full.append(tmp_df, ignore_index=True)
        
    except:
        print("읽으려는 파일의 포멧을 확인해야함, 파일명: ", a_file,)


Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)07.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)06.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)02.csv
Reading the file:  ./datasets/20220920_csv/02 해운대_배수펌프(3075794).csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)05.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation).xlsx
읽으려는 파일의 포멧을 확인해야함, 파일명:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation).xlsx
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)04.csv
Reading the file:  ./datasets/20220920_csv/병원_triple.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)03.csv
Reading the file:  ./datasets/20220920_csv/00 general_triple.csv
Reading the file:  ./datasets/20220920_csv/소방서_triple.csv
Reading the file:  ./datasets/20220920_csv/01 부산_유수

/tmp/ipykernel_2808834/4084913395.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kg_full = kg_full.append(a_df, ignore_index=True)
/tmp/ipykernel_2808834/4084913395.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full = df_full.append(tmp_df, ignore_index=True)
/tmp/ipykernel_2808834/4084913395.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kg_full = kg_full.append(a_df, ignore_index=True)
/tmp/ipykernel_2808834/4084913395.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full = df_full.append(tmp_df, ignore_index=True)
/tmp/ipykernel_2808834/4084913395.py:22: FutureWarning: The frame.append method is deprecate

In [108]:
kg_full

,head,relation,tail,file_name
0,7633,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
1,7634,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
2,7635,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
3,7636,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
4,7637,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
...,...,...,...,...
10438,광남지구대,latitude,35.14537016,경찰서_triple.csv
10439,망미2치안센터,latitude,35.1717912,경찰서_triple.csv
10440,부산사상경찰서,latitude,35.17452651,경찰서_triple.csv
10441,부산기장경찰서,latitude,35.32481808,경찰서_triple.csv


In [109]:
class Config:
    kge_n_epoch = 100
    kge_lr = 1e-4
    kge_batch = 64
    kge_margin = 0.5
    kge_conv_size = 3

In [110]:
config = Config()

In [111]:
#kg_full[["head", "relation", "tail"]].to_csv("triple_kb2.csv")
kg_full = kg_full.astype(str)

In [112]:
kb_emb_path = './kge_save'
os.makedirs(kb_emb_path, exist_ok=True)

In [113]:
#from torchkge.utils.datasets import load_fb15k
#kg, kg_val, kg_test = load_fb15k()
#
#model_convKB = ConvKBModel(256,
#                            3,
#                            kg.n_ent,
#                            kg.n_rel,
#)

In [114]:
kg, model_convKB = train_kge(kg_full[["head", "relation", "tail"]],
                             epochs=config.kge_n_epoch, 
                                lr=config.kge_lr, 
                                batch_size=config.kge_batch,
                                margin=config.kge_margin,
                                conv_size=config.kge_conv_size
                                )

originally counts 10443

user nunique 1250, item nunique 2457


Epoch 100 | mean loss: 4.59009: 100%|██████████| 100/100 [00:42<00:00,  2.34epoch/s]


In [117]:
emb_entity_ = model_convKB.get_embeddings()[0].detach().cpu().numpy()
emb_rel_ = model_convKB.get_embeddings()[1].detach().cpu().numpy()

ent_emb_dim = len(emb_entity_[0])
ent_vocab={}
ent_vocab['<unk>'] = 0
ent_vocab['<pad>'] = 1
ent_vocab.update({word: i+2 for word, i in kg.ent2ix.items()})
ent_ids = {i: word for word, i in ent_vocab.items()}

pad_emb_npa = np.zeros((1, ent_emb_dim))   #embedding for '<pad>' token.
unk_emb_npa = np.zeros((1, ent_emb_dim))   #embedding for '<unk>' token.

ent_embs_npa = np.vstack((pad_emb_npa,unk_emb_npa,emb_entity_))
print(ent_embs_npa.shape)
ent_emb = torch.nn.Embedding.from_pretrained(torch.from_numpy(ent_embs_npa).float(), freeze=False)

with open('ent_vocab_npa.npy','wb') as f:
    np.save(f,ent_vocab)

with open('ent_embs_npa.npy','wb') as f:
    np.save(f,ent_emb)


(3705, 256)


In [118]:
print(ent_emb(torch.LongTensor([10, 15]))) 
ids = ent_vocab["부곡지구대"]
print(ent_emb(torch.LongTensor([ids])))

tensor([[-2.3495e-02,  2.7541e-02, -1.4692e-02,  2.0852e-02, -5.6799e-03,
         -1.0114e-02,  6.6786e-03, -1.5817e-02,  1.3240e-02,  2.5674e-02,
          1.9184e-02,  5.0760e-03,  1.0856e-02, -6.8166e-03, -2.4069e-02,
          1.1869e-02, -4.7383e-03, -2.3067e-02,  3.5151e-02, -1.6735e-03,
          2.8682e-03,  1.4233e-02,  1.6200e-02, -5.2617e-03,  2.8609e-02,
         -2.6667e-03, -3.2197e-02, -3.7139e-03,  1.2010e-02, -7.6127e-04,
         -3.1458e-02, -1.9014e-03,  2.7984e-02,  3.5050e-02,  1.6103e-02,
          1.8406e-02, -1.9635e-02, -9.8370e-03,  2.9551e-02,  2.7809e-02,
          8.2627e-03,  5.5127e-03, -1.5847e-02, -3.5770e-02, -5.8009e-03,
         -7.3087e-03,  3.2922e-02, -2.6567e-02,  9.9924e-03, -5.5123e-03,
          4.5479e-03, -1.7145e-02,  1.5799e-02,  7.1761e-03, -1.3002e-02,
          1.8460e-02,  5.3515e-03, -2.5622e-02,  2.4684e-02,  2.0875e-03,
         -5.6280e-03,  3.3184e-02,  1.1812e-02, -2.4457e-02, -1.3043e-02,
          1.3301e-02,  2.3331e-02,  2.

In [119]:
ent_name="부곡지구대"
e1_ids = ent_vocab[ent_name]
e1_torch = ent_emb(torch.LongTensor([e1_ids]))
e2_ids = np.arange(0, len(ent_vocab))
e2_torch = ent_emb(torch.LongTensor([e2_ids])).squeeze()
similarity = cosine_similarity(e1_torch, e2_torch)
s_value, s_ids = torch.topk(similarity, 100)
s_ids = s_ids.squeeze()
s_ids

tensor([3018, 2912, 3278, 2862, 2921, 3263, 2893, 3312, 2928, 3202, 3161, 3159,
        3166, 3572, 3171, 2965, 3306, 3413, 3285, 2993, 3563, 3674, 3269, 2846,
        2970, 3310, 3164, 3308, 3173, 2864, 3243, 3185, 2888, 3248, 3163, 2920,
        3017, 2852, 2984, 2858, 3661, 2840, 2849, 3250, 3313, 3567,  363, 3128,
        3157, 2978, 3637, 3638,  976, 2949, 2889, 3326, 3153, 2821, 2973, 3165,
        2929, 2983, 3587, 3019, 3302, 3152, 3585, 2860, 3464, 2822, 3253, 3701,
        3016, 2881, 2861, 3634, 2979, 2863, 3148, 3314, 3149, 3011, 2823, 3247,
        3158, 2977, 3571, 3244, 2969, 3415, 3439, 3409, 2966, 3251, 3373, 2986,
        2609, 3283, 3411, 3414])

In [120]:
for idx in s_ids:
    print(idx)
    print(ent_ids[int(idx)])

tensor(3018)
부곡지구대
tensor(2912)
녹산파출소
tensor(3278)
신평파출소
tensor(2862)
구덕지구대
tensor(2921)
다대해양파출소
tensor(3263)
수성지구대
tensor(2893)
낙민파출소
tensor(3312)
온천3파출소
tensor(2928)
대연치안센터
tensor(3202)
사직지구대
tensor(3161)
부산사상경찰서
tensor(3159)
부산북부경찰서
tensor(3166)
부산연제경찰서
tensor(3572)
충렬지구대
tensor(3171)
부산지방경찰청경찰특공대
tensor(2965)
동래봉생병원
tensor(3306)
영선2치안센터
tensor(3413)
좌3치안센터
tensor(3285)
연산119안전센터
tensor(2993)
반송파출소
tensor(3563)
창선119안전센터
tensor(3674)
해운대경찰서
tensor(3269)
수영삼성병원
tensor(2846)
경희병원
tensor(2970)
동의병원
tensor(3310)
온종합병원
tensor(3164)
부산성소병원
tensor(3308)
영주파출소
tensor(3173)
부산지방경찰청제2기동대
tensor(2864)
구포부민병원
tensor(3243)
서울아동병원
tensor(3185)
부산진소방서
tensor(2888)
기장소방서
tensor(3248)
성지지구대
tensor(3163)
부산성모병원
tensor(2920)
다대자연병원
tensor(3017)
봉생병원
tensor(2852)
공항파출소
tensor(2984)
문현지구대
tensor(2858)
광혜병원
tensor(3661)
해동병원
tensor(2840)
개금1치안센터
tensor(2849)
고신의료원
tensor(3250)
세웅병원
tensor(3313)
온천지구대
tensor(3567)
청학119안전센터
tensor(363)
119수상구조대
tensor(3128)
부산강서경찰서
tensor(3157)
부산보훈병원
tensor(2978)
망미119안전

In [121]:
def similarity_example_on_TransE(kg, ent_name="부곡지구대"):
    e1 = emb_entity_[kg.ent2ix[ent_name]]
    r1 = emb_entity_[kg.rel2ix['koad:roadNameCode']]
    e2 = emb_entity_[kg.head_idx]
    similarity = cosine_similarity(torch.Tensor([e1]), torch.Tensor(e2))
    s_value, s_ids = torch.topk(similarity, 10)
    for idx in s_ids:
        print(int(idx))
        print(kg.head_idx[int(idx)])
        print(kg.ix2ent[int(kg.head_idx[int(idx)])])

## text to KB entity module
 - input: text (entity name)
 - output: 

In [122]:
class KLDiveEmb(nn.Module):
    def __init__(self, input_dim, hidden_dim, out_dim):
        super(KLDiveEmb, self).__init__()

        self.bert = transformers.XLMRobertaModel.from_pretrained('xlm-roberta-base')
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.drop_out = nn.Dropout(0.33)
        self.fc1 = nn.Linear(hidden_dim, out_dim)

    def forward(self, idx, mask):
    
        x = self.bert(idx, mask) 
        x = x.pooler_output
        x = F.relu(self.layer1(x))
        x = self.drop_out(x)
        x = F.relu(self.fc1(x))
        
        return F.softmax(x, dim=1), x

In [123]:
class KLEDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, data, max_token):
        
        self.tokenizer = tokenizer
        self.data = data
        self.max_token = max_token 
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        e_name = self.data['e_name'][index] #Entity Name
        e_emb = self.data['e_emb'][index]  #KB embedding
            
        tokenized = self.tokenizer.encode_plus("".join(e_name),
                                                None,
                                                add_special_tokens=True,
                                                max_length = self.max_token,
                                                padding='max_length',
                                                truncation=True,
                                              )
        
        ids = tokenized['input_ids']
        mask = tokenized['attention_mask']

        return {'e_ids': torch.tensor(ids, dtype=torch.long), 
                'e_mask': torch.tensor(mask, dtype=torch.long),
                'target': torch.tensor(e_emb)}

In [124]:
model = KLDiveEmb(768, 512, 256).to(DEVICE)
criterion = torch.nn.KLDivLoss(reduction='batchmean')
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005)
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [125]:
k,i,v = [],[],[]
for ent, idx in ent_vocab.items():
    ent_name=ent
    e1_ids = ent_vocab[ent_name]
    e1_torch = ent_emb(torch.LongTensor([e1_ids]))
    k.append(ent)
    i.append(idx)
    v.append(e1_torch.squeeze().detach().numpy())

In [126]:
df = pd.DataFrame(columns=['e_name','e_idx', 'e_emb'])
df["e_name"] = k
df["e_idx"] = i
df["e_emb"] = v

In [127]:
train_df, valid_df = train_test_split(df, test_size=0.2)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
batch_size = 64

In [128]:
train_dataset = KLEDataset(tokenizer, train_df, max_token=20)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=4, shuffle=True, pin_memory=True)
valid_dataset = KLEDataset(tokenizer, valid_df, max_token=20) 
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size , num_workers=4, shuffle=False, pin_memory=True)

In [129]:
def train_fn(model, train_loader, optimizer, criterion):
    model.train()
    batch_loss = []
    for idx, batch in tqdm(enumerate(train_loader), total=len(train_loader), leave=False): #학습 데이터를 batch size만큼씩 읽어옴
        optimizer.zero_grad()
        e_ids = batch['e_ids'].to(DEVICE) #entity_name
        e_bert_mask = batch['e_mask'].to(DEVICE) 
        
        logits, _ = model(e_ids, e_bert_mask) 
                
        target = batch['target'].to(DEVICE) #KB embedding
        target = F.softmax(target, dim=1)
        target.requires_grad = False
        loss = criterion(logits.log(), target)

        batch_loss.append(float(loss.item()))
        loss.backward(loss)
        optimizer.step()
        
    return sum(batch_loss) / len(train_loader)

In [130]:
def valid_fn(model, valid_loader, criterion):
    model.eval()
    batch_loss = []
    with torch.no_grad():
        for idx, batch in tqdm(enumerate(valid_loader), total=len(valid_loader), leave=False): #학습 데이터를 batch size만큼씩 읽어옴
            optimizer.zero_grad()
            e_ids = batch['e_ids'].to(DEVICE) #entity_name
            e_bert_mask = batch['e_mask'].to(DEVICE) 
            
            logits, _ = model(e_ids, e_bert_mask) 
                    
            target = batch['target'].to(DEVICE) #KB embedding
            target = F.softmax(target, dim=1)
            loss = criterion(logits.log(), target)

            batch_loss.append(float(loss.item()))
        
    return sum(batch_loss) / len(valid_loader)

In [131]:
train_epoch_loss = []
test_epoch_loss = []
for epoch in range(5):
    train_avg_loss = train_fn(model, train_loader, optimizer, criterion)
    train_epoch_loss.append(train_avg_loss)
    print(train_avg_loss)

0.0010727933085365022


0.0007536372583498188


0.000720258436829565


0.000706694227583865


0.0006988779146303522


In [132]:
valid_avg_loss = valid_fn(model, valid_loader, criterion)
print(valid_avg_loss)

0.0006897887845601266


In [133]:
def prediction(model, keyword="춘천"):

    e_name = keyword
    tokenized = tokenizer.encode_plus("".join(e_name),
                                        None,
                                        add_special_tokens=True,
                                        max_length = 20,
                                        padding='max_length',
                                        truncation=True,
                                        )

    ids = tokenized['input_ids']
    mask = tokenized['attention_mask']

    batch = {'e_ids': torch.tensor(ids, dtype=torch.long), 
             'e_mask': torch.tensor(mask, dtype=torch.long)}

    e_ids = batch['e_ids'].to(DEVICE).unsqueeze(0) #entity_name
    e_bert_mask = batch['e_mask'].to(DEVICE).unsqueeze(0) 
    logits, word_emb = model(e_ids, e_bert_mask)
    e1_torch = word_emb.cpu()
    e2_ids = np.arange(0, len(ent_vocab))
    e2_torch = ent_emb(torch.LongTensor([e2_ids])).squeeze()
    similarity = cosine_similarity(e1_torch, e2_torch)
    s_value, s_ids = torch.topk(similarity, 10)
    s_ids = s_ids.squeeze()
    s_ids

    return s_ids.detach().numpy(), s_value.detach().numpy()

In [134]:
def search_ex_matching(df, query = "<unk>"):
    keyword = query
    exist_boolean = (df["e_name"] == keyword).any()
    ex_matching = 0 #기본적으로 "<unk>"로 해둠
    if exist_boolean: 
        ex_matching = int(df[df["e_name"] == keyword]['e_idx']) 
    else: 
        ex_matching = 0

    return ex_matching

In [135]:
def levenshtein_sim(df, query):
    #idx = df["e_name"].apply(lambda x: levenshtein(x, query)).idxmin()
    idx = df["e_name"].apply(lambda x: jamo_levenshtein(x, query)).idxmin()
    return idx

In [136]:
def graph_ent_sim(ent_vocab, ent_emb, query, topN=5):
    ent_name=query
    e1_ids = ent_vocab[ent_name]
    e1_torch = ent_emb(torch.LongTensor([e1_ids]))
    e2_ids = np.arange(0, len(ent_vocab))
    e2_torch = ent_emb(torch.LongTensor([e2_ids])).squeeze()
    similarity = cosine_similarity(e1_torch, e2_torch)
    s_value, s_ids = torch.topk(similarity, topN)
    s_ids = s_ids.squeeze()
    return s_ids

In [139]:
query = "병원"

#EXmatching 결과
ex_ids = search_ex_matching(df, query)
ex_sim_query = df[df['e_idx'] == int(ex_ids)]["e_name"].iloc[0]

#Levenshtein 유사도 결과
lev_ids = levenshtein_sim(df, query)
lev_sim_query = df[df['e_idx'] == int(lev_ids)]["e_name"].iloc[0]

print("---------- Graph similarity 유사도 결과 ----------")
if ex_ids != 0:
    graph_ids = graph_ent_sim(ent_vocab, ent_emb, ex_sim_query)
else:
    graph_ids = graph_ent_sim(ent_vocab, ent_emb, lev_sim_query)


for idx in graph_ids:
    print("유사 Entity: ", idx, " ", df.iloc[int(idx)]["e_name"])
print("---------- Graph similarity 유사도 끝 ----------")
print("")
print("")


#BERT-Entity 유사도 결과
print("---------- BERT-Entity 유사도 결과 시작 ----------")
s_ids, s_score = prediction(model, query)
total_ids = s_ids #np.concatenate([np.array([ex_ids]), np.array([lev_ids]), s_ids]) 
for seq_num, idx in enumerate(total_ids):
    print(df[df['e_idx'] == int(idx)][["e_idx","e_name"]].values.tolist())
print("---------- BERT-Entity 유사도 결과 끝 ----------")


---------- Graph similarity 유사도 결과 ----------
유사 Entity:  tensor(3562)   참병원
유사 Entity:  tensor(3278)   신평파출소
유사 Entity:  tensor(2912)   녹산파출소
유사 Entity:  tensor(2921)   다대해양파출소
유사 Entity:  tensor(3312)   온천3파출소
---------- Graph similarity 유사도 끝 ----------


---------- BERT-Entity 유사도 결과 시작 ----------
[[1749, '7653']]
[[1784, '7688']]
[[2177, '8051']]
[[3167, '부산영도경찰서']]
[[2601, '8460']]
[[2427, '8294']]
[[1853, '7754']]
[[2099, '7995']]
[[3657, '한빛병원']]
[[2653, '8511']]
---------- BERT-Entity 유사도 결과 끝 ----------
